# 3 Nonlinear Equations and Complementarity Problems 

In [1]:

import numpy as np
from numpy import append, array, diagonal, tril, triu
from numpy.linalg import inv
from scipy.linalg import lu
#from scipy.linalg import solve
from pprint import pprint
from numpy import array, zeros, diag, diagflat, dot

from sympy import *
import sympy as sym
init_printing()

import matplotlib as mpl
# matplotlib for ploting
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D     # 3d
# for inline interactive plotting
%matplotlib notebook

#mpl.rcParams['savefig.dpi'] = 80
mpl.rcParams['figure.dpi'] = 80
# from IPython.display import set_matplotlib_formats
# set_matplotlib_formats('png', 'pdf')
# for better picture quality
%config InlineBackend.figure_format = 'retina'
#https://www.dataquest.io/blog/jupyter-notebook-tips-tricks-shortcuts/

import seaborn as sns
sns.set()
#sns.set_style(style= "whitegrid")
#plt.style.available
# bold graph style
plt.style.use('fivethirtyeight')

ModuleNotFoundError: No module named 'seaborn'

### Example: Monopoly

To illustrate the role of competition on the size of a market, the price of a good and firms' profits, let's start by considering a simple monopoly. Then we will increase the number of firms that are operating in the market.

In every case, we use the same inverse demand function for the good, which is $P(q)=\left( \dfrac{1}{q} \right)^{\epsilon}$


In [2]:
# Begin with a monopolist
qm = 1.0 # initial point/guess quantity
cm = 0.7 # cost
eta = 1.6 # elasticity 
e = -1/eta #Note: e=-epsilon; using e instead of epsilon simplifies the formulas

In [3]:
#Check that it works (Note: everything that follows is a number)
fvalm = qm**e + e*qm**(e-1)*qm - cm*qm # function value (foc)
fjacm = e*qm**(e-1)+e*qm**(e-1)+(e-1)*e*qm**(e-2)*qm-cm # function jacobian (derivative, in one dimension)
qm = qm - fvalm/fjacm # newton update for the quantity
qm

### Newton's method in one dimension

In [4]:
#Let's find the solution (i.e. the root of the foc)
maxit = 1000 # maximum number of iterations
tol = 10e-10 # tolerance level for convergence
for it in np.arange(maxit):

    fvalm = qm**e + e*qm**(e-1)*qm - cm*qm # function value (foc)
    fjacm = e*qm**(e-1)+e*qm**(e-1)+(e-1)*e*qm**(e-2)*qm-cm # function jacobian (derivative, in one dimension)
    qm = qm - fvalm/fjacm # newton update for the quantity
    if abs(fvalm)<tol:
        break
        
print('The final iteration is:',it)
print('The foc value is f:',fvalm)
print('The computed quantity is q:',qm)

#Final computations
pm=qm**e #price
pim=pm*qm-cm*qm**2 #profits

print('The price is p:',pm)
print('The profits are pi:',pim)
print('The market size is:',qm)

The final iteration is: 3
The foc value is f: 2.7200464103316335e-15
The computed quantity is q: 0.6810667503986678
The price is p: 1.2713246007441796
The profits are pi: 0.5411605715817014
The market size is: 0.6810667503986678


### Example: Cournot Duopoly

Now consider a simple Cournot duopoly model, with symmetrc firms, and in which the inverse demand for a good is 	$P(q_{1},q_{2})=\left( \dfrac{1}{q_{1}+q_{2}} \right)^{\epsilon}$

In [5]:
# Example from the textbook
#q = np.array([1]) # initial point/guess quantity
#c = np.array([0.7]) # cost vector
#eta = 1.6 # elasticity 
#e = -1/eta #Note: e=-epsilon; using e instead of epsilon simplifies the formulas
#nfirm=1

In [6]:
# Cournot example. One update (first step) of newton algorithm 
#fval = np.sum(q)**e + e * np.sum(q)**(e-1)*(q) - np.diag(c).dot(q) # function value
#fjac = e*np.sum(q)**(e-1)*(np.ones([nfirm,nfirm]))+e*np.sum(q)**(e-1)*(np.eye(nfirm))+(e-1)*e*np.sum(q)**(e-2)*(q).dot(np.array([1]))-np.diag(c) # function jacbian
#q = q - np.linalg.inv(fjac).dot(fval) # newton update for the quantity
#q

In [7]:
# Example from the textbook
q = np.array([1, 1]) # initial point/guess quantity
c = np.array([0.7, 0.7]) # cost vector
eta = 1.6 # elasticity 
e = -1/eta #Note: e=-epsilon; using e instead of epsilon simplifies the formulas

In [8]:
# Cournot example. One update (first step) of newton algorithm 
fval = np.sum(q)**e + e * np.sum(q)**(e-1)*(q) - np.diag(c).dot(q) # function value
fjac = e*np.sum(q)**(e-1)*(np.ones([2,2]))+e*np.sum(q)**(e-1)*(np.eye(2))+\
(e-1)*e*np.sum(q)**(e-2)*(q).dot(np.array([1, 1]))-np.diag(c) # function jacbian
q = q - np.linalg.inv(fjac).dot(fval) # newton update for the quantity
q

array([ 0.60850935,  0.60850935])

In [9]:
# first step result of objective value, and it should go to zero gradually
fval

array([-0.2542114, -0.2542114])

In [10]:
# first step result of jacobian
fjac

array([[-0.77598669,  0.12664449],
       [ 0.12664449, -0.77598669]])

### Newton's method in more than one dimension

In [11]:
# Example from the textbook
# This executes a finite numer of Newton iterations
maxit = 1000 # maximum number of iterations
tol = 10e-10 # tolerance level for convergence
# starting from [0.2 0.2] will cause a convergence problem (Newton's method has limitations)
q = np.array([1, 1])
c = np.array([0.7, 0.7])
eta = 1.6
e = -1/eta #Note: e=-epsilon; using e instead of epsilon simplifies the formulas
nfirm=2 #duopoly case (used only to compute the total market size)

for it in np.arange(maxit):

    fval = np.sum(q)**e + e * np.sum(q)**(e-1)*(q) - np.diag(c).dot(q)
    fjac = e*np.sum(q)**(e-1)*(np.ones([2,2]))+e*np.sum(q)**(e-1)*(np.eye(2))+(e-1)*e*np.sum(q)**(e-2)*(q).dot(np.array([1, 1]))-np.diag(c)
    q = q - np.linalg.inv(fjac).dot(fval)
    if np.linalg.norm(fval)<tol:
        break
        
print('The final iteration is:',it)
print('The foc value is f:',fval)
print('The computed quantity is q:',q)

#Final computations
pd=np.sum(q)**e #price
pid=pd*q[0]-c[0]*q[0]**2 #profits

print('The price is p:',pd)
print('The total profits are pi:',pid*nfirm)
print('The market size is:',np.sum(q))


The final iteration is: 108
The foc value is f: [ -6.65637989e-10  -6.65637989e-10]
The computed quantity is q: [ 0.75753663  0.75753663]
The price is p: 0.771310028062
The total profits are pi: 0.365184752316
The market size is: 1.51507326782


### Example: Multi-firm

Now consider a general Multi-firm model, with $I>1$ symmetric firms, and in which the inverse demand for a good is $P(q)=\left( \dfrac{1}{q} \right)^{\epsilon}$ and the aggregate quantity is $q=q_{1}+q_{2}+...+q_{I}=\sum_{i=1}^{I}q_{i}$

In [17]:
# Let's extend the example from the textbook to a many firms set-up
# This executes a finite numer of Newton iterations
maxit = 1000 # maximum number of iterations
tol = 10e-10 # tolerance level for convergence
eta = 1.6
e = -1/eta
nfirm=2 #this allows us to select the number of firms (used to solve the system of foc's)
#nfirm=5 #increased competition
#nfirm=1000 #close to perfect competition

#Initializations of the vectors of quantities and costs: we use lists and later convert them to arrays
qlist = []
clist = []

#These alternative initializations don't work necessarily
#qmf=np.array([])
#cmf=np.array([])
#qmf = ones(nfirm)

#Let's assign the desired initial guesses and vector of costs
for it in np.arange(nfirm):
    qlist.append(1.0)
    clist.append(0.7)

#Create a vector of ones and convert the lists to arrays for matematical manipulation
vecone=np.array(qlist)
qmf=np.array(qlist)
qmf=qmf*0.2
cmf=np.array(clist)
    
print('q1:',qmf[0])
#print('q2:',qmf[1])
print('c1:',cmf[0])
#print('c2:',cmf[1])

it=0

for it in np.arange(maxit):

    fvalmf = np.sum(qmf)**e + e * np.sum(qmf)**(e-1)*(qmf) - np.diag(cmf).dot(qmf)
    fjacmf = e*np.sum(qmf)**(e-1)*(np.ones([nfirm,nfirm]))+e*np.sum(qmf)**(e-1)*(np.eye(nfirm))+\
    (e-1)*e*np.sum(qmf)**(e-2)*(qmf).dot(np.eye(nfirm))-np.diag(cmf)
    #(e-1)*e*np.sum(qmf)**(e-2)*(qmf).dot(vecone)-np.diag(cmf)
    qmf = qmf - np.linalg.inv(fjacmf).dot(fvalmf)
    if np.linalg.norm(fvalmf)<tol:
        break
        
print('The final iteration is:',it)
print('The foc value is f:',fvalmf)
print('The computed quantity is q:',qmf)

#Final computations
pmf=np.sum(qmf)**e #price
pimf=pmf*qmf[0]-cmf[0]*qmf[0]**2 #profits

print('The price is p:',pmf)
print('The total profits are pi:',pimf*nfirm)
print('The market size is:',np.sum(qmf))

q1: 0.2
c1: 0.7
The final iteration is: 5
The foc value is f: [  1.93134397e-12   1.93134397e-12]
The computed quantity is q: [ 0.75753663  0.75753663]
The price is p: 0.771310027752
The total profits are pi: 0.365184751564
The market size is: 1.5150732688
